At the top of every petrify project is usually the same list of imports:

In [1]:
from petrify import u, tau, Point, Vector

You can find more detailed information for all of these classes in our [online documentation](https://petrify.readthedocs.io/en/latest/).

## Construction

Of course, the humble [`Box`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Box) is the literal building block of many pieces of solid geometry. You can use the mouse to interact with the camera in each of these examples:

In [2]:
from petrify.solid import Box

Box(Point.origin, Vector(1, 1, 1)).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

Its round cousin, the [`Cylinder`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Cylinder), is also frequently useful:

In [3]:
from petrify.solid import Cylinder

Cylinder(Point.origin, Vector.basis.y * 3, 1.0, segments=20).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

Another method for forming simple solids is to extrude a [`PlanarPolygon`](https://petrify.readthedocs.io/en/latest/petrify.space.html#petrify.space.PlanarPolygon) along a simple [`Vector`](https://petrify.readthedocs.io/en/latest/petrify.space.html#petrify.space.Vector) to form a [`PolygonExtrusion`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.PolygonExtrusion):

In [4]:
from petrify import Polygon
from petrify.solid import Basis, PlanarPolygon, PolygonExtrusion

triangle = Polygon([
    Point(0, 0),
    Point(0, 2),
    Point(1, 1)
])

PolygonExtrusion(PlanarPolygon(Basis.unit, triangle), Vector.basis.z).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

There are many advanced methods for spinning, extruding, and chaining planar polygons into complex solids. They are covered in detail in [another notebook](advanced.ipynb).

## Transformation

Every solid in petrify inherits from the Node base class. This class adds convenient functionality for performing  common operations including rotation, translation, and scaling:

In [5]:
from petrify.solid import Collection

original = Box(Point.origin, Vector(1, 1, 1))
translated = original + Vector.basis.y * 3
rotated = translated.rotate(Vector.basis.z, tau / 8)
scaled = translated.rotate(Vector.basis.z, tau / 4) * Vector(2, 2, 2)

Collection([
    original,
    translated.view(color='#0f0'),
    rotated.view(color='#f00'),
    scaled.view(color='#00f')
]).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

## Combination

Finally, perhaps the most common method for forming complex solids is the [`Union`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Union) of many smaller parts:

In [6]:
import random
from petrify.solid import Union

def spin(shape):
    return (
        shape
            .rotate(Vector.basis.x, random.uniform(0, tau / 16))
            .rotate(Vector.basis.y, random.uniform(0, tau / 16))
            .rotate(Vector.basis.z, random.uniform(0, tau / 16))
    )

def delta(): return random.uniform(0.75, 1.25)

cubes = [
    spin(Box(
        Point.origin,
        random.uniform(1, 2) * Vector(1, 1, 1)
    )) + Point(x * delta(), y * delta(), delta())
    for x in range(8)
    for y in range(3)
]

Union(cubes).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

This is a powerful feature which is covered in more detail in its [own example notebook](csg.ipynb).

## Import and Export

Defining all these cool solids is pretty useless without the ability to share them with the world. Fortunately, petrify supports exporting solids to [`STL`](https://petrify.readthedocs.io/en/latest/petrify.formats.html#petrify.formats.stl.STL) for slicers or other uses:

In [7]:
from petrify.formats import STL

cube = Box(Point.origin, Vector(1, 1, 1))
STL('cube.stl', u.mm).write(cube.as_unit(u.mm))

STL is a unitless format. You can read more about how and why petrify handles units in our [documentation](https://petrify.readthedocs.io/en/latest/petrify.units.html). The tl;dr is for STL files, you need to specify the units for both your geometry and the resulting file.

You can naturally also import STL-formatted files for manipulation within petrify:

In [8]:
cube = STL.read('xyz.stl', 'mm')
cube.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

petrify also has basic support for parsing paths from SVG files:

In [9]:
from petrify.formats import SVG
from petrify.space import Basis, PlanarPolygon

paths = SVG.read('logo.svg', u.inches / (90 * u.file))
logo = paths['logo'].m_as(u.inches)
logo_path = PlanarPolygon(Basis.xy, logo.polygon())
logo_path.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

Which can be directly extruded like any other polygon:

In [10]:
PolygonExtrusion(logo_path, Vector(0, 0, 0.25)).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…